In [15]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint
import os.path as op
import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torchaudio
import argparse
import fnmatch
import logging
import multiprocessing as mp
from typing import Dict, List, Tuple

import librosa
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [16]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


# MCD on train

In [17]:
split_txt_train_path = r'.\preprocessed_data\Ego4D_final_v6\train.txt'
train_uids = []
with open(split_txt_train_path) as file:
    for line in file:
        # print(line.split('|')[0])
        train_uids.append(line.split('|')[0])

In [18]:
len(train_uids)

27292

In [19]:
recon_folder = r'.\output\0703a\result\Ego4D_final_v6\train\wav\reconstructed'
syn_folder = r'.\output\0703a\result\Ego4D_final_v6\train\wav\synthesized'
gt_folder = r'.\raw_data\Ego4D_final_v6\train\Ego4D_final_v6'

In [21]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_recon = []
for uid in tqdm(train_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(recon_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_recon.append(result)

  0%|          | 0/27292 [00:00<?, ?it/s]

CPU times: total: 2h 23min 24s
Wall time: 2h 38min 25s


In [22]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_gt = []
for uid in tqdm(train_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_gt.append(result)

  0%|          | 0/27292 [00:00<?, ?it/s]

CPU times: total: 2h 19min 46s
Wall time: 2h 30min 16s


In [24]:
import pandas as pd
v6_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [25]:
v6_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0703a_1M4_train_new.csv")

In [26]:
print(f"MCDs_recon mean on 0703a_1M4_train: {torch.tensor(MCDs_recon).mean()}")
print(f"MCDs_recon std on 0703a_1M4_train: {torch.tensor(MCDs_recon).std()}")

print(f"MCDs_gt mean on 0703a_1M4_train: {torch.tensor(MCDs_gt).mean()}")
print(f"MCDs_gt std on 0703a_1M4_train: {torch.tensor(MCDs_gt).std()}")

MCDs_recon mean on 0703a_1M4_train: 6.216004815657436
MCDs_recon std on 0703a_1M4_train: 1.9483218658869703
MCDs_gt mean on 0703a_1M4_train: 7.510086980942805
MCDs_gt std on 0703a_1M4_train: 1.621730248844869


In [27]:
v6_mcd_df.nsmallest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
18705,628b9cba-d4e9-477a-b2cb-44207a9afd6c,3.022329,4.958812
11980,1009fdc3-d5fa-49a3-981b-f94d06a18e3f,3.217786,4.339374
20493,26dbb6c0-58ed-44e9-9b70-e556cc1b1802,3.234197,5.359722
10224,1f581e26-3aec-4f07-880f-a53dc14e4ea4,3.244544,4.674834
13950,5999682b-da38-4a18-a0b5-41ce5ccf4300,3.290473,4.888488


In [28]:
v6_mcd_df.nlargest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
12063,54bdedf1-45c0-495c-82ff-151d9e59c037,23.131395,23.283965
4272,54ffc6f5-6c59-481c-87e4-122e8dac82a9,23.008096,21.002178
3325,0dfe7ee4-945f-451c-ad09-0cfb8cf755cb,22.705428,22.235879
16451,0ca4e349-c203-476b-bdde-672f7074386d,22.557707,23.699070
12214,5f421cca-532f-4991-b094-1006b34a510e,21.926005,22.033782
